In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gregorut/videogamesales")

csv_path = path + "/vgsales.csv"

In [8]:
import polars as pl

vgsales = pl.read_csv(csv_path, null_values=["N/A"])
# Show the first 5 rows
print(vgsales.head())


shape: (5, 11)
┌──────┬──────────────────┬──────────┬──────┬───┬──────────┬──────────┬─────────────┬──────────────┐
│ Rank ┆ Name             ┆ Platform ┆ Year ┆ … ┆ EU_Sales ┆ JP_Sales ┆ Other_Sales ┆ Global_Sales │
│ ---  ┆ ---              ┆ ---      ┆ ---  ┆   ┆ ---      ┆ ---      ┆ ---         ┆ ---          │
│ i64  ┆ str              ┆ str      ┆ i64  ┆   ┆ f64      ┆ f64      ┆ f64         ┆ f64          │
╞══════╪══════════════════╪══════════╪══════╪═══╪══════════╪══════════╪═════════════╪══════════════╡
│ 1    ┆ Wii Sports       ┆ Wii      ┆ 2006 ┆ … ┆ 29.02    ┆ 3.77     ┆ 8.46        ┆ 82.74        │
│ 2    ┆ Super Mario      ┆ NES      ┆ 1985 ┆ … ┆ 3.58     ┆ 6.81     ┆ 0.77        ┆ 40.24        │
│      ┆ Bros.            ┆          ┆      ┆   ┆          ┆          ┆             ┆              │
│ 3    ┆ Mario Kart Wii   ┆ Wii      ┆ 2008 ┆ … ┆ 12.88    ┆ 3.79     ┆ 3.31        ┆ 35.82        │
│ 4    ┆ Wii Sports       ┆ Wii      ┆ 2009 ┆ … ┆ 11.01    ┆ 3.28     ┆ 2.96

In [9]:
print(vgsales.columns)
print(len(vgsales['Name'].unique()))
print(vgsales.shape)
print(vgsales.dtypes)



['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
11493
(16598, 11)
[Int64, String, String, Int64, String, String, Float64, Float64, Float64, Float64, Float64]


In [10]:
print([x + ", " + str(vgsales[x].is_null().sum()) for x in vgsales.columns])


['Rank, 0', 'Name, 0', 'Platform, 0', 'Year, 271', 'Genre, 0', 'Publisher, 58', 'NA_Sales, 0', 'EU_Sales, 0', 'JP_Sales, 0', 'Other_Sales, 0', 'Global_Sales, 0']


In [11]:
vgsales = vgsales.drop_nulls()

print(vgsales.columns)
print(vgsales.shape)
print(vgsales.dtypes)
print([x + ", " + str(vgsales[x].is_null().sum()) for x in vgsales.columns])
print(vgsales['Rank'].unique())


['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
(16291, 11)
[Int64, String, String, Int64, String, String, Float64, Float64, Float64, Float64, Float64]
['Rank, 0', 'Name, 0', 'Platform, 0', 'Year, 0', 'Genre, 0', 'Publisher, 0', 'NA_Sales, 0', 'EU_Sales, 0', 'JP_Sales, 0', 'Other_Sales, 0', 'Global_Sales, 0']
shape: (16_291,)
Series: 'Rank' [i64]
[
	1
	2
	3
	4
	5
	…
	16596
	16597
	16598
	16599
	16600
]


In [12]:
# load vào clickhouse

from clickhouse_driver import Client
client = Client(host='localhost', port=9000, user='default', password='secret')


client.execute(
    'INSERT INTO vgsales_stage.vgsales_raw (Rank, Name, Platform, Year, Genre, Publisher, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales) VALUES',
    vgsales.to_dicts()  # Chuyển DataFrame Polars thành list of dicts
)

16291